In [ ]:
# # combine sessionmaker Session, with-statement & begin() to start session, handle exceptions, rollback, commit and close at once
# with Session.begin() as session:
#     session.add(object)
#     session.add_all([objects])
#     result = session.query(Project).filter(Project.name == 'Test').scalar() # scalar is good if only one match is possible due to database constraints

In [1]:
import settings # type: ignore
from camtrappy.db.testdb import initialize_new_project
from camtrappy.io import parsing

if not 'initialized' in locals():
    # Set data Dir & Data Project (file parser)
    DATA_DIR = settings.data_dir_animals
    new_project = parsing.ProjectParser(name='Test', projectfolder=None, datafolder=DATA_DIR, restrict_to=[])

    # Create new db project with parsed data
    initialize_new_project(new_project)
    initialized = True

home:      n:\Mitarbeiter\Florian\projects\CamTrapPy
data:      n:\Mitarbeiter\Florian\projects\CamTrapPy\tests\data
notebooks: n:\Mitarbeiter\Florian\projects\CamTrapPy\notebooks
--Setup complete--
Parsing folder: 7_Ittlingen
Parsing folder: 21_Herrenberg
Parsing folder: 25_Stockach
Parsing folder: best
Parsing folder: Location1
Parsing folder: Location2
2021-08-04 15:03:55,257 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-04 15:03:55,257 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("projects")
2021-08-04 15:03:55,257 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-04 15:03:55,267 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("projects")
2021-08-04 15:03:55,267 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-04 15:03:55,267 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("locations")
2021-08-04 15:03:55,267 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-04 15:03:55,267 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("locations")
2021-08-04 1

In [2]:
import settings # type: ignore
from camtrappy.db.testdb import Session
from camtrappy.db.schema import Video, Location, Object, VideoObject
from camtrappy.core.base import VideoList, VideoLoader

with Session.begin() as session:
    q = session.query(Video)
r = q.limit(10).all()
v = r[0]
print(v.fullpath)

with Session.begin() as session:
    q = session.query(Video).where(Video.location.has(name='7_Ittlingen')).order_by(Video.date, Video.time)
r = q.all()
print(r)

with Session.begin() as session:
    o = Object()
    vo = VideoObject(video_id=1, object=o, frames=[1,2,3], bboxes=[[0, 10, 50, 50], [10, 20, 50, 50]])
    vo2 = VideoObject(video_id=2, object=o)
    session.add_all([vo, vo2])
    
with Session.begin() as session:
    q = session.query(VideoObject)
r = q.first()
print(r.video_id, r.object.id, r.frames, r.bboxes)

2021-08-04 15:04:05,287 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-04 15:04:05,291 INFO sqlalchemy.engine.Engine SELECT videos.id AS videos_id, videos.path AS videos_path, videos.date AS videos_date, videos.time AS videos_time, videos.fps AS videos_fps, videos.duration AS videos_duration, videos.date_added AS videos_date_added, videos.location_id AS videos_location_id, (SELECT projects.datafolder 
FROM projects, locations 
WHERE projects.id = locations.project_id AND locations.id = videos.location_id) AS anon_1, (SELECT locations.folder 
FROM locations 
WHERE locations.id = videos.location_id) AS anon_2, (SELECT projects.datafolder 
FROM projects, locations 
WHERE projects.id = locations.project_id AND locations.id = videos.location_id) || ? || (SELECT locations.folder 
FROM locations 
WHERE locations.id = videos.location_id) || ? || videos.path AS anon_3 
FROM videos
 LIMIT ? OFFSET ?
2021-08-04 15:04:05,292 INFO sqlalchemy.engine.Engine [generated in 0.00087s] ('\\', '\\'

# Gutes Setting

gb7 = GaussianBlur((7, 7))

ce = CannyEdge(180, 400)

re = Resize()

bgs_mog = BgsMOG(250, 5, 0.7, 25, -1)

norm = Normalize()

bi = BilateralFilter(9)

base_transforms = [re]

compare_transforms = [bi, norm, bgs_mog, gb7, ce]  # ce is not so good for findContours

In [1]:
import settings # type: ignore
from camtrappy.db.testdb import Session
from camtrappy.core.base import VideoLoader, VideoPlayer
from camtrappy.core.transforms import (CannyEdge, GaussianBlur, TransformFactory,
                                       AdaptiveHistogram, Resize, Gamma, Hysteresis,
                                       BGS, Normalize, BilateralFilter, AdaptiveThreshold,
                                       BgsMOG, Threshold)

from camtrappy.core.analysis import CentroidTracker

at = AdaptiveThreshold(7, 3)
ah = AdaptiveHistogram(2, (12, 12))
gb3 = GaussianBlur((3, 3))
ce = CannyEdge(180, 400)
gamma = Gamma(0.25)
hyst = Hysteresis(0.15, 0.20)
bgs1 = BGS('KNN', 0.1)
bgs2 = BGS('KNN', 0.8)


bi = BilateralFilter(9) # smoothing that keeps the edges
norm = Normalize() # a little bit homogenization
bgs_mog = BgsMOG(hist=250, nmixtures=5, background_ratio=0.7,
                 noise=25, learningrate=-1) # background subtractor
gb7 = GaussianBlur((7, 7)) # some more smoothing
thr = Threshold(50, 'tozero') # eliminate some more noise


transforms = TransformFactory([bi, norm, bgs_mog, gb7, thr])

vl = VideoLoader(Session, 3, skip_n_frames=4) #, transforms=transforms)
object_tracker = CentroidTracker(min_area=30, eps=1.5, maxDisappeared=50, Session=Session)
VideoPlayer(vl).play(resize=Resize(), transforms=transforms, visitor=object_tracker, compare=True)

home:      n:\Mitarbeiter\Florian\projects\CamTrapPy
data:      n:\Mitarbeiter\Florian\projects\CamTrapPy\tests\data
notebooks: n:\Mitarbeiter\Florian\projects\CamTrapPy\notebooks
--Setup complete--
2021-08-04 15:21:48,153 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-04 15:21:48,158 INFO sqlalchemy.engine.Engine SELECT videos.id AS videos_id, videos.path AS videos_path, videos.date AS videos_date, videos.time AS videos_time, videos.fps AS videos_fps, videos.duration AS videos_duration, videos.date_added AS videos_date_added, videos.location_id AS videos_location_id, (SELECT projects.datafolder 
FROM projects, locations 
WHERE projects.id = locations.project_id AND locations.id = videos.location_id) AS anon_1, (SELECT locations.folder 
FROM locations 
WHERE locations.id = videos.location_id) AS anon_2, (SELECT projects.datafolder 
FROM projects, locations 
WHERE projects.id = locations.project_id AND locations.id = videos.location_id) || ? || (SELECT locations.folder 
FROM loc